# Preprocessing: BJH026


This markdown files loads the clean data and does a bipolar rereference as well as epochs the data


In [1]:
import matplotlib
matplotlib.use("Qt5Agg")
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
from scipy import signal, stats
import mat73
import re
from neurodsp.timefrequency import compute_wavelet_transform
import os
import mne
import IPython
import seaborn as sns

In [2]:
## Prep paths ##

subject = 'BJH026'
raw_data_dir = f"/home/brooke/pacman/raw_data/{subject}"
preproc_data_dir = f"/home/brooke/pacman/preprocessing/{subject}/ieeg"

In [3]:
## Load Data ##

# load filtered data #
filtered_clean_fif = mne.io.Raw(f"{raw_data_dir}/ieeg/{subject}_notched_filtered_clean_ieeg.fif")

# load raw data #
raw_clean_fif = mne.io.Raw(f"{raw_data_dir}/ieeg/{subject}_raw_clean_ieeg.fif")



Opening raw data file /home/brooke/pacman/raw_data/BJH026/ieeg/BJH026_notched_filtered_clean_ieeg.fif...
    Range : 0 ... 2085999 =      0.000 ...  1042.999 secs
Ready.
Opening raw data file /home/brooke/pacman/raw_data/BJH026/ieeg/BJH026_notched_filtered_clean_ieeg-1.fif...
    Range : 2086000 ... 2920199 =   1043.000 ...  1460.100 secs
Ready.
Opening raw data file /home/brooke/pacman/raw_data/BJH026/ieeg/BJH026_raw_clean_ieeg.fif...
    Range : 0 ... 2085999 =      0.000 ...  1042.999 secs
Ready.
Opening raw data file /home/brooke/pacman/raw_data/BJH026/ieeg/BJH026_raw_clean_ieeg-1.fif...
    Range : 2086000 ... 2920199 =   1043.000 ...  1460.100 secs
Ready.


## Bipolar Rereferencing

In [4]:
# helper functions

def probe_and_num(elec_str): 
    ''' This convenience function takes an electrode string like LHH1 and outputs ('LHH', 1)
    '''
    if '_' in elec_str:
        return None, None
    else:   
        regex_str = '(\D+)(\d+)' # group of letters followed by group of digits
        matcher = re.compile(regex_str,re.IGNORECASE|re.DOTALL).search(elec_str)
        if matcher:
            probe, num = matcher.groups()
            return probe, int(num)
        else:
            return None, None
    
def find_bipolar_pair(ch, labels, remove):
    ''' Find the pair of a given electrode for bipolar referencing.
        Given a single *ch* and a list of *labels*, some of which you want to *remove*,
        this finds the next channel on that probe that isn't meant to be removed (WM is ok).
    '''
    bipolar_pair = None
    probe, num = probe_and_num(ch)
    other_in_probe = [other_ch for other_ch in labels if probe_and_num(other_ch)[0]==probe]
    for i in range(len(other_in_probe)-num):
        next_ch = probe + str(num+i+1)
        if next_ch in remove or next_ch == 'STI':
            continue
        else:
            bipolar_pair = next_ch
            break
    return bipolar_pair

In [5]:
# Here I iterate through the electrodes in my ROIs and match them up with their bipolar pair
pairs = []
anode = []
cathode = []
pairs_name = []
pairs_map = {}
remove = filtered_clean_fif.info['bads']
labels = filtered_clean_fif.info['ch_names']
for ch in labels:
    # if a channel is meant to be removed, it doesn't get to be in a bipolar pair
    if ch in remove or ch == 'STI':
        print(f"{ch} noref")
    else:
        pair = find_bipolar_pair(ch, labels, remove)
        if pair:
            anode.append(ch)
            cathode.append(pair)
            pairs.append((ch, pair))
            pairs_map[ch] = f"{ch}-{pair}"
            pairs_name.append(f"{ch}-{pair}")
            print(ch, pair)

EMPTY noref
EMPTY_2 noref
EMPTY_3 noref
EMPTY_4 noref
REF1 noref
REF2 noref
AL1 AL2
AL2 AL3
AL3 AL4
AL4 AL5
AL5 AL6
AL6 AL7
AL7 AL8
AL8 AL9
AL9 AL10
AL10 AL11
AL11 AL12
AL12 AL13
AL13 AL14
AL14 AL15
AL15 AL16
BL1 BL2
BL2 BL3
BL3 BL4
BL4 BL5
BL5 BL6
BL6 BL7
BL7 BL8
BL8 BL9
BL9 BL10
BL10 BL11
BL11 BL12
BL12 BL13
BL13 BL14
BL14 BL15
BL15 BL16
CL1 CL2
CL2 CL3
CL3 CL4
CL4 CL5
CL5 CL6
CL6 CL7
CL7 CL8
DL1 DL2
DL2 DL3
DL3 DL4
DL4 DL5
DL5 DL6
DL6 DL7
DL7 DL8
DL8 DL9
DL9 DL10
DL10 DL11
DL11 DL12
DL12 DL13
DL13 DL14
DL14 DL15
DL15 DL16
EMPTY_63 noref
EMPTY_64 noref
EL1 noref
EL2 noref
EL3 noref
EL4 noref
EL5 noref
EL6 noref
EL7 noref
EL8 noref
EL9 noref
EL10 noref
FL1 noref
FL2 noref
FL3 noref
FL4 noref
FL5 noref
FL6 noref
FL7 noref
FL8 noref
FL9 noref
FL10 noref
GL1 noref
GL2 noref
GL3 noref
GL4 noref
GL5 noref
GL6 noref
GL7 noref
GL8 noref
HL1 noref
HL2 noref
HL3 noref
HL4 noref
HL5 noref
HL6 noref
HL7 noref
HL8 noref
IL1 noref
IL2 noref
IL3 noref
IL4 noref
IL5 noref
IL6 noref
IL7 IL8
IL8 IL9
I

In [6]:
## Apply Rereference #

if filtered_clean_fif.info['ch_names'] == raw_clean_fif.info['ch_names'] and filtered_clean_fif.info['bads'] == raw_clean_fif.info['bads']:
    
    # load filtered data
    filtered_clean_fif.load_data()
    
    # set filtered reference 
    bp_filt_fif = mne.set_bipolar_reference(filtered_clean_fif, anode = anode, cathode = cathode)
    
    # save
    bp_filt_fif.save(f"{preproc_data_dir}/{subject}_bp_filtered_clean_data.fif", overwrite = True)
    
    # clear var
    bp_filt_fif = []
    filtered_clean_fif = []
    
    # load raw data
    raw_clean_fif.load_data()
    
    # set raw reference 
    bp_raw_fif = mne.set_bipolar_reference(raw_clean_fif, anode = anode, cathode = cathode)
    
    # save
    bp_raw_fif.save(f"{preproc_data_dir}/{subject}_bp_raw_clean_data.fif", overwrite = True)

    
    # clear var
    bp_raw_fif = []
    raw_clean_fif = []    
    
    

Reading 0 ... 2920199  =      0.000 ...  1460.100 secs...
sEEG channel type selected for re-referencing
Creating RawArray with float64 data, n_channels=89, n_times=2920200
    Range : 0 ... 2920199 =      0.000 ...  1460.100 secs
Ready.
Added the following bipolar channels:
AL1-AL2, AL2-AL3, AL3-AL4, AL4-AL5, AL5-AL6, AL6-AL7, AL7-AL8, AL8-AL9, AL9-AL10, AL10-AL11, AL11-AL12, AL12-AL13, AL13-AL14, AL14-AL15, AL15-AL16, BL1-BL2, BL2-BL3, BL3-BL4, BL4-BL5, BL5-BL6, BL6-BL7, BL7-BL8, BL8-BL9, BL9-BL10, BL10-BL11, BL11-BL12, BL12-BL13, BL13-BL14, BL14-BL15, BL15-BL16, CL1-CL2, CL2-CL3, CL3-CL4, CL4-CL5, CL5-CL6, CL6-CL7, CL7-CL8, DL1-DL2, DL2-DL3, DL3-DL4, DL4-DL5, DL5-DL6, DL6-DL7, DL7-DL8, DL8-DL9, DL9-DL10, DL10-DL11, DL11-DL12, DL12-DL13, DL13-DL14, DL14-DL15, DL15-DL16, IL7-IL8, IL8-IL9, IL9-IL10, IL10-IL11, IL11-IL12, JL9-JL10, JL10-JL11, JL11-JL12, KL1-KL2, KL2-KL3, KL3-KL4, KL4-KL5, KL5-KL6, KL6-KL7, KL7-KL8, KL8-KL9, KL9-KL10, KL10-KL11, KL11-KL12, LL1-LL2, LL2-LL3, LL3-LL4, LL4-L

/tmp/ipykernel_143097/1336304392.py:12: RuntimeWarning: This filename (/home/brooke/pacman/preprocessing/BJH026/ieeg/BJH026_bp_filtered_clean_data.fif) does not conform to MNE naming conventions. All raw files should end with raw.fif, raw_sss.fif, raw_tsss.fif, _meg.fif, _eeg.fif, _ieeg.fif, raw.fif.gz, raw_sss.fif.gz, raw_tsss.fif.gz, _meg.fif.gz, _eeg.fif.gz or _ieeg.fif.gz
  bp_filt_fif.save(f"{preproc_data_dir}/{subject}_bp_filtered_clean_data.fif", overwrite = True)


Overwriting existing file.
Writing /home/brooke/pacman/preprocessing/BJH026/ieeg/BJH026_bp_filtered_clean_data-1.fif
Closing /home/brooke/pacman/preprocessing/BJH026/ieeg/BJH026_bp_filtered_clean_data-1.fif
Closing /home/brooke/pacman/preprocessing/BJH026/ieeg/BJH026_bp_filtered_clean_data.fif
[done]
Reading 0 ... 2920199  =      0.000 ...  1460.100 secs...
sEEG channel type selected for re-referencing
Creating RawArray with float64 data, n_channels=89, n_times=2920200
    Range : 0 ... 2920199 =      0.000 ...  1460.100 secs
Ready.
Added the following bipolar channels:
AL1-AL2, AL2-AL3, AL3-AL4, AL4-AL5, AL5-AL6, AL6-AL7, AL7-AL8, AL8-AL9, AL9-AL10, AL10-AL11, AL11-AL12, AL12-AL13, AL13-AL14, AL14-AL15, AL15-AL16, BL1-BL2, BL2-BL3, BL3-BL4, BL4-BL5, BL5-BL6, BL6-BL7, BL7-BL8, BL8-BL9, BL9-BL10, BL10-BL11, BL11-BL12, BL12-BL13, BL13-BL14, BL14-BL15, BL15-BL16, CL1-CL2, CL2-CL3, CL3-CL4, CL4-CL5, CL5-CL6, CL6-CL7, CL7-CL8, DL1-DL2, DL2-DL3, DL3-DL4, DL4-DL5, DL5-DL6, DL6-DL7, DL7-DL8, D

/tmp/ipykernel_143097/1336304392.py:25: RuntimeWarning: This filename (/home/brooke/pacman/preprocessing/BJH026/ieeg/BJH026_bp_raw_clean_data.fif) does not conform to MNE naming conventions. All raw files should end with raw.fif, raw_sss.fif, raw_tsss.fif, _meg.fif, _eeg.fif, _ieeg.fif, raw.fif.gz, raw_sss.fif.gz, raw_tsss.fif.gz, _meg.fif.gz, _eeg.fif.gz or _ieeg.fif.gz
  bp_raw_fif.save(f"{preproc_data_dir}/{subject}_bp_raw_clean_data.fif", overwrite = True)


Overwriting existing file.
Writing /home/brooke/pacman/preprocessing/BJH026/ieeg/BJH026_bp_raw_clean_data-1.fif
Closing /home/brooke/pacman/preprocessing/BJH026/ieeg/BJH026_bp_raw_clean_data-1.fif
Closing /home/brooke/pacman/preprocessing/BJH026/ieeg/BJH026_bp_raw_clean_data.fif
[done]


In [7]:
# Visualize it #
bp_filtered_fif = mne.io.Raw(f"{preproc_data_dir}/{subject}_bp_filtered_clean_data.fif")
events = mne.find_events(bp_filtered_fif, output='step', consecutive = False, stim_channel='STI')
bp_filtered_fif.plot(events=events, color='b', bad_color = 'cyan', n_channels = 1, clipping = None, event_color = 'r')

Opening raw data file /home/brooke/pacman/preprocessing/BJH026/ieeg/BJH026_bp_filtered_clean_data.fif...
    Range : 0 ... 2161999 =      0.000 ...  1080.999 secs
Ready.
Opening raw data file /home/brooke/pacman/preprocessing/BJH026/ieeg/BJH026_bp_filtered_clean_data-1.fif...
    Range : 2162000 ... 2920199 =   1081.000 ...  1460.100 secs
Ready.


/tmp/ipykernel_143097/2239763971.py:2: RuntimeWarning: This filename (/home/brooke/pacman/preprocessing/BJH026/ieeg/BJH026_bp_filtered_clean_data.fif) does not conform to MNE naming conventions. All raw files should end with raw.fif, raw_sss.fif, raw_tsss.fif, _meg.fif, _eeg.fif, _ieeg.fif, raw.fif.gz, raw_sss.fif.gz, raw_tsss.fif.gz, _meg.fif.gz, _eeg.fif.gz or _ieeg.fif.gz
  bp_filtered_fif = mne.io.Raw(f"{preproc_data_dir}/{subject}_bp_filtered_clean_data.fif")


480 events found
Event IDs: [0 1]
Using qt as 2D backend.


Channels marked as bad:
['C3', 'C4', 'CZ', 'EKG1', 'EKG2', 'EL1', 'EL10', 'EL2', 'EL3', 'EL4', 'EL5', 'EL6', 'EL7', 'EL8', 'EL9', 'EMPTY', 'EMPTY_125', 'EMPTY_126', 'EMPTY_127', 'EMPTY_128', 'EMPTY_163', 'EMPTY_164', 'EMPTY_165', 'EMPTY_166', 'EMPTY_167', 'EMPTY_168', 'EMPTY_169', 'EMPTY_170', 'EMPTY_171', 'EMPTY_172', 'EMPTY_173', 'EMPTY_174', 'EMPTY_175', 'EMPTY_176', 'EMPTY_177', 'EMPTY_178', 'EMPTY_179', 'EMPTY_180', 'EMPTY_181', 'EMPTY_182', 'EMPTY_183', 'EMPTY_184', 'EMPTY_185', 'EMPTY_186', 'EMPTY_187', 'EMPTY_188', 'EMPTY_189', 'EMPTY_190', 'EMPTY_191', 'EMPTY_192', 'EMPTY_193', 'EMPTY_194', 'EMPTY_195', 'EMPTY_196', 'EMPTY_197', 'EMPTY_198', 'EMPTY_199', 'EMPTY_2', 'EMPTY_200', 'EMPTY_201', 'EMPTY_202', 'EMPTY_203', 'EMPTY_204', 'EMPTY_205', 'EMPTY_206', 'EMPTY_207', 'EMPTY_208', 'EMPTY_209', 'EMPTY_210', 'EMPTY_211', 'EMPTY_212', 'EMPTY_213', 'EMPTY_214', 'EMPTY_215', 'EMPTY_216', 'EMPTY_217', 'EMPTY_218', 'EMPTY_219', 'EMPTY_220', 'EMPTY_221', 'EMPTY_222', 'EMPTY_223', 'EMPT

## Epoching the data 

### Onset

In [8]:
## Epoching the raw data ##

# load data 
bp_filtered_fif = mne.io.Raw(f"{preproc_data_dir}/{subject}_bp_filtered_clean_data.fif")
events = mne.find_events(bp_filtered_fif, output='step', consecutive = False, stim_channel='STI')

# create events
    
# only epoch good channels
channels_of_interest =  [x for x in bp_filtered_fif.info['ch_names'] if x not in bp_filtered_fif.info['bads']]

# epoch the data
epoched_data = mne.Epochs(bp_filtered_fif, events, 
                          event_id = 1, tmin = -4, tmax = 12, 
                          baseline = None, picks = channels_of_interest,
                             reject_by_annotation = False)
# save the data
epoched_data.save(f"{preproc_data_dir}/{subject}_bp_clean_pres-locked_ieeg.fif", overwrite = True, split_size = '1.9GB')

Opening raw data file /home/brooke/pacman/preprocessing/BJH026/ieeg/BJH026_bp_filtered_clean_data.fif...
    Range : 0 ... 2161999 =      0.000 ...  1080.999 secs
Ready.
Opening raw data file /home/brooke/pacman/preprocessing/BJH026/ieeg/BJH026_bp_filtered_clean_data-1.fif...
    Range : 2162000 ... 2920199 =   1081.000 ...  1460.100 secs
Ready.


/tmp/ipykernel_143097/1463149468.py:4: RuntimeWarning: This filename (/home/brooke/pacman/preprocessing/BJH026/ieeg/BJH026_bp_filtered_clean_data.fif) does not conform to MNE naming conventions. All raw files should end with raw.fif, raw_sss.fif, raw_tsss.fif, _meg.fif, _eeg.fif, _ieeg.fif, raw.fif.gz, raw_sss.fif.gz, raw_tsss.fif.gz, _meg.fif.gz, _eeg.fif.gz or _ieeg.fif.gz
  bp_filtered_fif = mne.io.Raw(f"{preproc_data_dir}/{subject}_bp_filtered_clean_data.fif")


480 events found
Event IDs: [0 1]
Not setting metadata
240 matching events found
No baseline correction applied
0 projection items activated
Loading data for 1 events and 32001 original time points ...
Splitting into 2 parts
Loading data for 120 events and 32001 original time points ...


/tmp/ipykernel_143097/1463149468.py:18: RuntimeWarning: This filename (/home/brooke/pacman/preprocessing/BJH026/ieeg/BJH026_bp_clean_pres-locked_ieeg.fif) does not conform to MNE naming conventions. All epochs files should end with -epo.fif, -epo.fif.gz, _epo.fif or _epo.fif.gz
  epoched_data.save(f"{preproc_data_dir}/{subject}_bp_clean_pres-locked_ieeg.fif", overwrite = True, split_size = '1.9GB')


Loading data for 120 events and 32001 original time points ...


### Trial End

In [10]:
## Epoching the raw data ##

# load data 
bp_filtered_fif = mne.io.Raw(f"{preproc_data_dir}/{subject}_bp_filtered_clean_data.fif")
events = mne.find_events(bp_filtered_fif, output='step', consecutive = False, stim_channel='STI')

# create events
events = events[0:-1]

# only epoch good channels
channels_of_interest =  [x for x in bp_filtered_fif.info['ch_names'] if x not in bp_filtered_fif.info['bads']]

# epoch the data
epoched_data = mne.Epochs(bp_filtered_fif, events, 
                          event_id = 0, tmin = -5, tmax = 5, 
                          baseline = None, picks = channels_of_interest,
                             reject_by_annotation = False)
# save the data
epoched_data.save(f"{preproc_data_dir}/{subject}_bp_clean_end-locked_ieeg.fif", overwrite = True, split_size = '1.9GB')

Opening raw data file /home/brooke/pacman/preprocessing/BJH026/ieeg/BJH026_bp_filtered_clean_data.fif...
    Range : 0 ... 2161999 =      0.000 ...  1080.999 secs
Ready.
Opening raw data file /home/brooke/pacman/preprocessing/BJH026/ieeg/BJH026_bp_filtered_clean_data-1.fif...
    Range : 2162000 ... 2920199 =   1081.000 ...  1460.100 secs
Ready.


/tmp/ipykernel_143097/875914644.py:4: RuntimeWarning: This filename (/home/brooke/pacman/preprocessing/BJH026/ieeg/BJH026_bp_filtered_clean_data.fif) does not conform to MNE naming conventions. All raw files should end with raw.fif, raw_sss.fif, raw_tsss.fif, _meg.fif, _eeg.fif, _ieeg.fif, raw.fif.gz, raw_sss.fif.gz, raw_tsss.fif.gz, _meg.fif.gz, _eeg.fif.gz or _ieeg.fif.gz
  bp_filtered_fif = mne.io.Raw(f"{preproc_data_dir}/{subject}_bp_filtered_clean_data.fif")


480 events found
Event IDs: [0 1]
Not setting metadata
239 matching events found
No baseline correction applied
0 projection items activated
Overwriting existing file.
Loading data for 1 events and 20001 original time points ...
Loading data for 239 events and 20001 original time points ...


/tmp/ipykernel_143097/875914644.py:19: RuntimeWarning: This filename (/home/brooke/pacman/preprocessing/BJH026/ieeg/BJH026_bp_clean_end-locked_ieeg.fif) does not conform to MNE naming conventions. All epochs files should end with -epo.fif, -epo.fif.gz, _epo.fif or _epo.fif.gz
  epoched_data.save(f"{preproc_data_dir}/{subject}_bp_clean_end-locked_ieeg.fif", overwrite = True, split_size = '1.9GB')


### Last Away

In [11]:
## Load Behavioral Data ##
last_away_data = pd.read_csv(f"{raw_data_dir}/behave/{subject}_last_away_events.csv")
last_away_data

,neural_trial_numeric,TrialType,sample,sample_before,event
0,0,13,26600,0,1
1,1,20,52200,0,1
2,2,6,75700,0,1
3,3,7,84500,0,1
4,5,14,120500,0,1
...,...,...,...,...,...
200,234,14,2828200,0,1
201,235,8,2838400,0,1
202,236,12,2846900,0,1
203,237,10,2856900,0,1


In [12]:
# create events
last_away_events = last_away_data[['sample', 'sample_before', 'event']].copy().to_numpy()


In [13]:
# load data 
bp_filtered_fif = mne.io.Raw(f"{preproc_data_dir}/{subject}_bp_filtered_clean_data.fif")
    
# only epoch good channels
channels_of_interest =  [x for x in bp_filtered_fif.info['ch_names'] if x not in bp_filtered_fif.info['bads']]

# epoch the data
last_away_epochs = mne.Epochs(bp_filtered_fif, last_away_events, 
                          event_id = 1, tmin = -5, tmax =5, 
                          baseline = None, picks = channels_of_interest,
                             reject_by_annotation = False)
# # save the data
last_away_epochs.save(f"{preproc_data_dir}/{subject}_bp_filtered_clean_last_away_events.fif", overwrite = True, split_size = '1.9GB')


Opening raw data file /home/brooke/pacman/preprocessing/BJH026/ieeg/BJH026_bp_filtered_clean_data.fif...
    Range : 0 ... 2161999 =      0.000 ...  1080.999 secs
Ready.
Opening raw data file /home/brooke/pacman/preprocessing/BJH026/ieeg/BJH026_bp_filtered_clean_data-1.fif...
    Range : 2162000 ... 2920199 =   1081.000 ...  1460.100 secs
Ready.
Not setting metadata
205 matching events found
No baseline correction applied
0 projection items activated
Loading data for 1 events and 20001 original time points ...
Loading data for 205 events and 20001 original time points ...


/tmp/ipykernel_143097/678285560.py:2: RuntimeWarning: This filename (/home/brooke/pacman/preprocessing/BJH026/ieeg/BJH026_bp_filtered_clean_data.fif) does not conform to MNE naming conventions. All raw files should end with raw.fif, raw_sss.fif, raw_tsss.fif, _meg.fif, _eeg.fif, _ieeg.fif, raw.fif.gz, raw_sss.fif.gz, raw_tsss.fif.gz, _meg.fif.gz, _eeg.fif.gz or _ieeg.fif.gz
  bp_filtered_fif = mne.io.Raw(f"{preproc_data_dir}/{subject}_bp_filtered_clean_data.fif")
/tmp/ipykernel_143097/678285560.py:13: RuntimeWarning: This filename (/home/brooke/pacman/preprocessing/BJH026/ieeg/BJH026_bp_filtered_clean_last_away_events.fif) does not conform to MNE naming conventions. All epochs files should end with -epo.fif, -epo.fif.gz, _epo.fif or _epo.fif.gz
  last_away_epochs.save(f"{preproc_data_dir}/{subject}_bp_filtered_clean_last_away_events.fif", overwrite = True, split_size = '1.9GB')


### First Dot

In [14]:
## Load Behavioral Data ##
first_dot_data = pd.read_csv(f"{raw_data_dir}/behave/{subject}_first_dot_events.csv")
first_dot_data

,neural_trial_numeric,TrialType,sample,sample_before,event
0,0,13,21700,0,1
1,1,20,46100,0,1
2,2,6,69800,0,1
3,4,3,97100,0,1
4,5,14,113500,0,1
...,...,...,...,...,...
226,234,14,2826200,0,1
227,235,8,2836800,0,1
228,236,12,2845800,0,1
229,237,10,2855900,0,1


In [15]:
# create events
first_dot_data = first_dot_data[['sample', 'sample_before', 'event']].copy().to_numpy()


In [17]:
# load data 
bp_filtered_fif = mne.io.Raw(f"{preproc_data_dir}/{subject}_bp_filtered_clean_data.fif")

# only epoch good channels
channels_of_interest =  [x for x in bp_filtered_fif.info['ch_names'] if x not in bp_filtered_fif.info['bads']]


# epoch the data
first_dot_epochs = mne.Epochs(bp_filtered_fif, first_dot_data, 
                          event_id = 1, tmin = -3, tmax =6, 
                          baseline = None, picks = channels_of_interest,
                             reject_by_annotation = False)
# # save the data
first_dot_epochs.save(f"{preproc_data_dir}/{subject}_bp_filtered_clean_first_dot_events.fif", overwrite = True, split_size = '1.9GB')


Opening raw data file /home/brooke/pacman/preprocessing/BJH026/ieeg/BJH026_bp_filtered_clean_data.fif...
    Range : 0 ... 2161999 =      0.000 ...  1080.999 secs
Ready.
Opening raw data file /home/brooke/pacman/preprocessing/BJH026/ieeg/BJH026_bp_filtered_clean_data-1.fif...
    Range : 2162000 ... 2920199 =   1081.000 ...  1460.100 secs
Ready.
Not setting metadata
231 matching events found
No baseline correction applied
0 projection items activated
Overwriting existing file.
Loading data for 1 events and 18001 original time points ...


/tmp/ipykernel_143097/4126832509.py:2: RuntimeWarning: This filename (/home/brooke/pacman/preprocessing/BJH026/ieeg/BJH026_bp_filtered_clean_data.fif) does not conform to MNE naming conventions. All raw files should end with raw.fif, raw_sss.fif, raw_tsss.fif, _meg.fif, _eeg.fif, _ieeg.fif, raw.fif.gz, raw_sss.fif.gz, raw_tsss.fif.gz, _meg.fif.gz, _eeg.fif.gz or _ieeg.fif.gz
  bp_filtered_fif = mne.io.Raw(f"{preproc_data_dir}/{subject}_bp_filtered_clean_data.fif")
/tmp/ipykernel_143097/4126832509.py:14: RuntimeWarning: This filename (/home/brooke/pacman/preprocessing/BJH026/ieeg/BJH026_bp_filtered_clean_first_dot_events.fif) does not conform to MNE naming conventions. All epochs files should end with -epo.fif, -epo.fif.gz, _epo.fif or _epo.fif.gz
  first_dot_epochs.save(f"{preproc_data_dir}/{subject}_bp_filtered_clean_first_dot_events.fif", overwrite = True, split_size = '1.9GB')


Loading data for 231 events and 18001 original time points ...


## First Move

In [18]:
## Load Behavioral Data ##
first_move_data = pd.read_csv(f"{raw_data_dir}/behave/{subject}_first_move_events.csv")
first_move_data

,neural_trial_numeric,TrialType,sample,sample_before,event
0,0,13,18300,0,1
1,1,20,43500,0,1
2,2,6,66000,0,1
3,3,7,84500,0,1
4,4,3,97000,0,1
...,...,...,...,...,...
234,234,14,2826000,0,1
235,235,8,2836700,0,1
236,236,12,2845300,0,1
237,237,10,2855500,0,1


In [19]:
# create events
first_move_data = first_move_data[['sample', 'sample_before', 'event']].copy().to_numpy()


In [20]:
# load data 
bp_filtered_fif = mne.io.Raw(f"{preproc_data_dir}/{subject}_bp_filtered_clean_data.fif")

# only epoch good channels
channels_of_interest =  [x for x in bp_filtered_fif.info['ch_names'] if x not in bp_filtered_fif.info['bads']]


# epoch the data
first_move_epochs = mne.Epochs(bp_filtered_fif, first_move_data, 
                          event_id = 1, tmin = -5, tmax =5, 
                          baseline = None, picks = channels_of_interest,
                             reject_by_annotation = False)
# # save the data
first_move_epochs.save(f"{preproc_data_dir}/{subject}_bp_filtered_clean_first_move_events.fif", overwrite = True, split_size = '1.9GB')


Opening raw data file /home/brooke/pacman/preprocessing/BJH026/ieeg/BJH026_bp_filtered_clean_data.fif...
    Range : 0 ... 2161999 =      0.000 ...  1080.999 secs
Ready.
Opening raw data file /home/brooke/pacman/preprocessing/BJH026/ieeg/BJH026_bp_filtered_clean_data-1.fif...
    Range : 2162000 ... 2920199 =   1081.000 ...  1460.100 secs
Ready.
Not setting metadata
239 matching events found
No baseline correction applied
0 projection items activated
Loading data for 1 events and 20001 original time points ...
Loading data for 239 events and 20001 original time points ...


/tmp/ipykernel_143097/3347570552.py:2: RuntimeWarning: This filename (/home/brooke/pacman/preprocessing/BJH026/ieeg/BJH026_bp_filtered_clean_data.fif) does not conform to MNE naming conventions. All raw files should end with raw.fif, raw_sss.fif, raw_tsss.fif, _meg.fif, _eeg.fif, _ieeg.fif, raw.fif.gz, raw_sss.fif.gz, raw_tsss.fif.gz, _meg.fif.gz, _eeg.fif.gz or _ieeg.fif.gz
  bp_filtered_fif = mne.io.Raw(f"{preproc_data_dir}/{subject}_bp_filtered_clean_data.fif")
/tmp/ipykernel_143097/3347570552.py:14: RuntimeWarning: This filename (/home/brooke/pacman/preprocessing/BJH026/ieeg/BJH026_bp_filtered_clean_first_move_events.fif) does not conform to MNE naming conventions. All epochs files should end with -epo.fif, -epo.fif.gz, _epo.fif or _epo.fif.gz
  first_move_epochs.save(f"{preproc_data_dir}/{subject}_bp_filtered_clean_first_move_events.fif", overwrite = True, split_size = '1.9GB')


## Ghost Attack

In [21]:
## Load Behavioral Data ##
ghost_attack_data = pd.read_csv(f"{raw_data_dir}/behave/{subject}_attack_events.csv")
ghost_attack_data

,neural_trial_numeric,TrialType,sample,sample_before,event
0,2,6,74500,0,1
1,4,3,99900,0,1
2,6,15,128100,0,1
3,7,5,138500,0,1
4,9,10,164700,0,1
...,...,...,...,...,...
59,223,13,2707600,0,1
60,226,16,2741700,0,1
61,227,4,2753800,0,1
62,234,14,2827800,0,1


In [22]:
# create events
ghost_attack_data = ghost_attack_data[['sample', 'sample_before', 'event']].copy().to_numpy()


In [24]:
# load data 
bp_filtered_fif = mne.io.Raw(f"{preproc_data_dir}/{subject}_bp_filtered_clean_data.fif")

# only epoch good channels
channels_of_interest =  [x for x in bp_filtered_fif.info['ch_names'] if x not in bp_filtered_fif.info['bads']]


# epoch the data
ghost_attack_epochs = mne.Epochs(bp_filtered_fif, ghost_attack_data, 
                          event_id = 1, tmin = -4, tmax =6, 
                          baseline = None, picks = channels_of_interest,
                             reject_by_annotation = False)
# # save the data
ghost_attack_epochs.save(f"{preproc_data_dir}/{subject}_bp_filtered_clean_ghost_attack_events.fif", overwrite = True, split_size = '1.9GB')


Opening raw data file /home/brooke/pacman/preprocessing/BJH026/ieeg/BJH026_bp_filtered_clean_data.fif...
    Range : 0 ... 2161999 =      0.000 ...  1080.999 secs
Ready.
Opening raw data file /home/brooke/pacman/preprocessing/BJH026/ieeg/BJH026_bp_filtered_clean_data-1.fif...
    Range : 2162000 ... 2920199 =   1081.000 ...  1460.100 secs
Ready.
Not setting metadata
64 matching events found
No baseline correction applied
0 projection items activated
Overwriting existing file.
Loading data for 1 events and 20001 original time points ...


/tmp/ipykernel_143097/2681655913.py:2: RuntimeWarning: This filename (/home/brooke/pacman/preprocessing/BJH026/ieeg/BJH026_bp_filtered_clean_data.fif) does not conform to MNE naming conventions. All raw files should end with raw.fif, raw_sss.fif, raw_tsss.fif, _meg.fif, _eeg.fif, _ieeg.fif, raw.fif.gz, raw_sss.fif.gz, raw_tsss.fif.gz, _meg.fif.gz, _eeg.fif.gz or _ieeg.fif.gz
  bp_filtered_fif = mne.io.Raw(f"{preproc_data_dir}/{subject}_bp_filtered_clean_data.fif")
/tmp/ipykernel_143097/2681655913.py:14: RuntimeWarning: This filename (/home/brooke/pacman/preprocessing/BJH026/ieeg/BJH026_bp_filtered_clean_ghost_attack_events.fif) does not conform to MNE naming conventions. All epochs files should end with -epo.fif, -epo.fif.gz, _epo.fif or _epo.fif.gz
  ghost_attack_epochs.save(f"{preproc_data_dir}/{subject}_bp_filtered_clean_ghost_attack_events.fif", overwrite = True, split_size = '1.9GB')


Loading data for 64 events and 20001 original time points ...
